# AI-FW2 効果測定1 問題1

google colaboratoryに"kouka_1_1_data.zip"をアップロードし、下記のセルを実行してデータを解凍してください

In [ ]:
!unzip "kouka_1_1_data.zip"

## 要求事項(1)

In [2]:
# ライブラリのインポート
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from sklearn import datasets, svm, metrics
from sklearn.metrics import accuracy_score
import pickle

画像ファイルを1次元のndarrayに変換する関数を準備しておく

In [3]:
# * 引数 filename ... 画像ファイルのパスを受け取る
# * 戻り値 ... 画像を1次元配列にした結果を返す
def image2array(filename):
    # 自分で用意した手書きの画像ファイルを読み込む
    my_img = cv2.imread(filename)

    # 画像データを学習用に加工する

    # 画像をグレースケールにする
    my_img = cv2.cvtColor(my_img, cv2.COLOR_BGR2GRAY)
    
    # 画像を32×32にリサイズする(使用する画像は元から32×32の大きさだが処理を記載すること)
    my_img = cv2.resize(my_img, (32, 32))
    
    # 画像を白黒反転する
    my_img = 15 - my_img // 16
    
    # 二次元を一次元に変換する(データ数は32×32個となる)
    my_img = my_img.reshape((-1, 32*32))

    return my_img

特徴量X、正解データyのデータを作成する

In [4]:
# ファイルの一覧(index_train.csv)を読み込み、データフレームを作成する
df_index = pd.read_csv('index_train.csv')
print(df_index.head())

# 読み込んだデータフレームのファイル名で画像ファイルを読み込み、特徴量Xに代入する
# 空のデータフレームを作成しておく
X = pd.DataFrame()

# forで1レコードずつ取り出す
for index, row in df_index.iterrows():
    # 作成したimage2array関数を使い、画像を一次元配列に変換する
    ret = image2array(row['filename'])

    # 変換後の配列(ndarray型)からDataFrameを生成し、特徴量Xに追加する
    X = X.append(pd.DataFrame(ret))

# 読み込んだデータフレームのtarget列を、正解データyに代入する
y = df_index['target']

                   filename target
0  ./traindata/61_00000.png      a
1  ./traindata/61_00001.png      a
2  ./traindata/61_00002.png      a
3  ./traindata/61_00003.png      a
4  ./traindata/61_00004.png      a


学習の実行と評価

In [5]:
# 特徴量X、正解データyをデータを学習用とテスト用に分割する
# テスト用データは全体の20%とする
# 乱数シードは0とする
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

# SVCのモデルを生成する
# 乱数シードは0とする
clf = svm.SVC(random_state=0)

# 学習用データを学習する
clf.fit(X_train, y_train)

# テスト用データで予測して精度を確認する
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.8317535545023697


▶実行結果  
0.8317535545023697

In [6]:
# 学習済みモデルを「kouka_1_model_氏名pkl」として保存する
with open('kouka_1_model_stu.pkl', 'wb') as fp:
    pickle.dump(clf, fp)

## 要求事項(2)

In [7]:
# ファイルの一覧取得で使用するライブラリをインポートする
import glob
from natsort import natsorted

In [8]:
# 必要に応じて学習済みモデルを読み込む
with open('sample_model.pkl', 'rb') as fp:
    clf = pickle.load(fp)

In [9]:
# ファイルの一覧をファイル名の昇順で取得する
files = natsorted(glob.glob('./predictdata/*.png'))

# CSVに書き込むためのデータフレームを作成数
result = pd.DataFrame(columns=['file','predict'])

# ファイルの一覧から1つずファイルを取り出し、予測する
for file in files:

    # 作成したimage2array関数を使い、画像を一次元配列に変換する
    my_image = image2array(file)
    
    # データ予測する
    res = clf.predict(my_image)

    # データフレームに追加するデータをディクショナリ型で格納する。
    # 予測結果は配列ではく、結果のみの文字列とする。
    append_data = {'file': file, 'predict':res[0]}
    result = result.append([append_data])

# resultの内容をCSVファイルに保存する。indexはつけないようにする
result.to_csv('result.csv',index=False)